In [ ]:
from dotenv import load_dotenv
from os import getenv
load_dotenv()

In [ ]:
!ls -l train_model.py

In [ ]:
!dstat --provider google-v2 --project shalek-lab --users 'ben' --status '*' --age 1d # --names "rf-bclls" --full

# FGID

In [ ]:
# FGID 
clusts = {
    0: "Tclls",
    1: "Bclls", 
    2: "Epith", 
    3: "Mloid", 
    4: "IGA_IGK_Plsma", 
#     5: "Fibro", 
    6: "IGA_IGL_Plsma", 
    7: "Mstcl", 
    8: "Endth", 
#     9: "Glial", 
    11: "IGG_Plsma_Bcll"
}

In [ ]:
clusts

In [ ]:
BUCKET = getenv("PREDICT_BUCKET") # PREDICT STUDY bucket

In [ ]:
for k, v in clusts.items():
    !dsub \
        --name RF_{v} \
        --provider google-v2 \
        --project shalek-lab \
        --logging gs://{BUCKET}/Tiered-FGID-13/logging \
        --regions us-central1 \
        --boot-disk-size 15  --timeout 4d \
        --min-ram 256 --min-cores 16 --disk-size 500 \
        --input LOOMFILE=gs://{BUCKET}/Tiered-FGID-13/loomfiles/T1C{k}_SCT_data.loom \
        --input METAFILE=gs://{BUCKET}/Tiered-FGID-13/20200630_clmeta_FG13_all.tsv \
        --input PARAMFILE=gs://{BUCKET}/Tiered-FGID-13/randomforest/20200421_randomforest_params.yml \
        --output OUTMODEL=gs://{BUCKET}/Tiered-FGID-13/randomforest/curated_models/FGID_T1C{k}_{v}_SCT_data.pkl \
        --output OUTRESULT=gs://{BUCKET}/Tiered-FGID-13/randomforest/curated_results/FGID_T1C{k}_{v}_SCT_data.csv \
        --env CLASSCOLUMN="curatedname" \
        --image shaleklab/terra-seurat-env:release-v2.0 \
        --script train_model.py

In [ ]:
!dstat --provider google-v2 --project shalek-lab --users 'ben' --status '*' --age 1d 

# CD

In [ ]:
# CD 
clusts = {  
    0: "Tclls",
    1: "Bclls",
    # 2: "Epith",
    3: "Mloid",
    4: "IGA_IGK_Plsma",
    5: "Fibro",
    6: "IGG_Plsma",
    7: "IGA_IGL_Plsma",
    8: "Mstcl",
    9: "Endth",
#     10: "IGM_Plsma",
#     11: "Paneth"
}

In [ ]:
clusts

In [ ]:
BUCKET = getenv("PREDICT_BUCKET") # PREDICT STUDY bucket

In [ ]:
for k, v in clusts.items():
    !dsub \
        --name cd-rf-{v} \
        --provider google-v2 \
        --project shalek-lab \
        --logging gs://{BUCKET}/Tiered-CD-14/logging \
        --regions us-central1 \
        --boot-disk-size 15  --timeout 4d \
        --min-ram 256 --min-cores 16 --disk-size 500 \
        --input LOOMFILE=gs://{BUCKET}/Tiered-CD-14/loomfiles/T1C{k}_SCT_data.loom \
        --input METAFILE=gs://{BUCKET}/Tiered-CD-14/20200630_clmeta_CD14_all.tsv \
        --input PARAMFILE=gs://{BUCKET}/Tiered-FGID-13/randomforest/20200421_randomforest_params.yml \
        --output OUTMODEL=gs://{BUCKET}/Tiered-CD-14/randomforest/curated_models/CD_T1C{k}_{v}_SCT_data.pkl \
        --output OUTRESULT=gs://{BUCKET}/Tiered-CD-14/randomforest/curated_results/CD_T1C{k}_{v}_SCT_data.csv \
        --env CLASSCOLUMN="curatedname" \
        --image shaleklab/terra-seurat-env:release-v2.0 \
        --script train_model.py

In [ ]:
!dstat --provider google-v2 --project shalek-lab --users 'ben' --status --age 12h # --names "rf-bclls" --full

# Weirdo groups

In [ ]:
!dsub \
    --name fgid-fibroglial \
    --provider google-v2 \
    --project shalek-lab \
    --logging gs://{BUCKET}/Tiered-FGID-13/logging \
    --regions us-central1 \
    --boot-disk-size 15  --timeout 4d \
    --min-ram 128 --min-cores 16 --disk-size 500 \
    --input LOOMFILE=gs://{BUCKET}/Tiered-FGID-13/loomfiles/FGID_fibro_glial.loom \
    --input METAFILE=gs://{BUCKET}/Tiered-FGID-13/20200630_clmeta_FG13_all.tsv \
    --input PARAMFILE=gs://{BUCKET}/Tiered-FGID-13/randomforest/20200421_randomforest_params.yml \
    --output OUTMODEL=gs://{BUCKET}/Tiered-FGID-13/randomforest/curated_models/FGID_fibro_glial_SCT_data.pkl \
    --output OUTRESULT=gs://{BUCKET}/Tiered-FGID-13/randomforest/curated_results/FGID_fibro_glial_SCT_data.csv \
    --env CLASSCOLUMN="curatedname" \
    --image shaleklab/terra-seurat-env:release-v2.0 \
    --script train_model.py

In [ ]:
!dsub \
        --name cd-epithpaneth \
        --provider google-v2 \
        --project shalek-lab \
        --logging gs://{BUCKET}/Tiered-CD-14/logging \
        --regions us-central1 \
        --boot-disk-size 15  --timeout 4d \
        --min-ram 128 --min-cores 16 --disk-size 500 \
        --input LOOMFILE=gs://{BUCKET}/Tiered-CD-14/loomfiles/CD_epith_paneth.loom \
        --input METAFILE=gs://{BUCKET}/Tiered-CD-14/20200630_clmeta_CD14_all.tsv \
        --input PARAMFILE=gs://{BUCKET}/Tiered-FGID-13/randomforest/20200421_randomforest_params.yml \
        --output OUTMODEL=gs://{BUCKET}/Tiered-CD-14/randomforest/curated_models/CD_epith_paneth_SCT_data.pkl \
        --output OUTRESULT=gs://{BUCKET}/Tiered-CD-14/randomforest/curated_results/CD_epith_paneth_SCT_data.csv \
        --env CLASSCOLUMN="curatedname" \
        --image shaleklab/terra-seurat-env:release-v2.0 \
        --script train_model.py